# Capture RGBD 2D Snapshots from 3D Scenes and Project them Back to 3D

TBD.

In [1]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [18]:
def visualize_obj_file(obj_path: str) -> None:
    mesh = o3d.io.read_triangle_mesh(obj_path, enable_post_processing=True)  # Load the OBJ file with TEXTURE!
    if mesh.is_empty():
        raise ValueError(f"Failed to load the OBJ file at {obj_path}. The mesh is empty.")

    o3d.visualization.draw_geometries([mesh]) 


In [19]:
# Path to the OBJ file
obj_file = "../models/ikea_cup_reality_scan_iphone/Cup.obj"

# Visualize the OBJ
visualize_obj_file(obj_file)